In [ ]:
import sys
sys.path.insert(0, "..")
sys.path

In [ ]:
from yolo.models import YOLOv1
from yolo.utils.voc_preprocess import get_bboxes
from utils.img_process import show_bbox

import torch

if __name__ == "__main__":
    test = torch.randn(3, 448, 448).unsqueeze(0)
    yolo = YOLOv1()
    res = yolo(test)

    # output: 7 * 7 * 30
    # S * S * ((x, y, w, h, confidence) * B=2 + C=20)
    bbox_pth = "/home/ubuntu/VOC2007/VOCdevkit/VOC2007/Annotations/000048.xml"
    bboxes = get_bboxes(bbox_pth)
    for b in bboxes:
        show_bbox("/home/ubuntu/VOC2007/VOCdevkit/VOC2007/JPEGImages/000048.jpg", b)